In [1]:
import sys
sys.path.append('/dls_sw/i13-1/scripts/ptyrex/ptyrex_3/')
import ptyrex
import json
from ptyrex import np
from ptyrex import h5py

ImportError: No module named 'cv2'


2020-06-19 10:05:09,008:/dls_sw/apps/python/anaconda/4.6.14/64/envs/python3.7/lib/python3.7/site-packages/matplotlib/pyplot.py:225:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


Set paths

In [2]:
#base_dir = '/dls/e02/data/2020/cm26481-1/processing/Merlin/20200130_80kV_graphene_600C_pty/20200131 143302/'
#data_id = '20200131_143302'
#data_filename = '80kV_600C_CLA_20um_CL_8cm_8C_20Mx_A2_4p71_dfneg70nm_scan_array_255by255_diff_plane_515by515_.hdf5'

base_dir = '/dls/e02/data/2019/cm22979-6/processing/Merlin/Merlin/20191001_15kV_ptycho/MoS2_700C/20191002 160536/'
expt_id = 'AuGe'
data_id = '20191002_160536'
data_filename = 'AuonGe_15kV_100umClAp_3Mx_def77_A2_1p07scan_array_253by255_diff_plane_515by515_.hdf5'

raw_data_path = base_dir + data_filename

Populate meta data file

In [3]:
pty_expt = {}

pty_expt['institute'] = 'dls'
pty_expt['instrument'] = 'e02'
pty_expt['datetime'] = ''
pty_expt['expt_id'] = expt_id
pty_expt['data_id'] = data_id

pty_expt['source'] = {}
pty_expt['source']['radiation'] = 'electron'
pty_expt['source']['energy'] = 15e3

pty_expt['detector'] = {}
pty_expt['detector']['name'] = 'merlin'
pty_expt['detector']['bit_depth'] = 12
pty_expt['detector']['shape'] = [512, 512]
pty_expt['detector']['pixel_pitch'] = [55e-6, 55e-6]
pty_expt['detector']['position'] = [0, 0, 0.187]

pty_expt['scan'] = {}
pty_expt['scan']['element'] = 'beam'
pty_expt['scan']['type'] = 'tv'
pty_expt['scan']['fast_axis'] = 1
pty_expt['scan']['step'] = [2.6e-10, 2.6e-10]
pty_expt['scan']['shape'] = [253, 255]
pty_expt['scan']['path'] = ''

pty_expt['data'] = {}
pty_expt['data']['path'] = raw_data_path
pty_expt['data']['key'] = 'Experiments/__unnamed__/data'

pty_expt['optics'] = {}
pty_expt['optics']['type'] = 'lens'
pty_expt['optics']['alpha'] = 0.017
pty_expt['optics']['defocus'] = -30e-09

In [4]:
with open(base_dir + 'pty_meta_' + data_id + '.json','w') as f:
    json.dump(pty_expt, f, indent=4)

Create scan position file

In [5]:
scan = ptyrex.core.model.scan.Scan()
scan.type = pty_expt['scan']['type']
scan.step = pty_expt['scan']['step']
scan.sz = pty_expt['scan']['shape']
scan.fast_axis = pty_expt['scan']['fast_axis']
scan.create()
scan.positions *= 1e6 # currently expects ums, should be all SI units eventually.
print("Scan list shape:", scan.positions.shape)

2020-06-19 10:05:09,134:/dls_sw/i13-1/scripts/ptyrex/ptyrex_3/ptyrex/core/loader/scan.py:7:Generating scan..


Scan list shape: (64515, 2)


Convert 4D data to 3D

In [6]:
with h5py.File(pty_expt['data']['path'],'r') as f:
    data_full = np.array(f[pty_expt['data']['key']])
print("4D shape:", data_full.shape)
data_full = np.reshape(data_full,[np.prod(data_full.shape[:2]), data_full.shape[-2], data_full.shape[-1]])
print("3D shape:", data_full.shape)

4D shape: (253, 255, 515, 515)
3D shape: (64515, 515, 515)


In [8]:
data_key = 'data/frames'
scan_key = 'data/scan'
filename = base_dir + 'pty_data_'+ data_id +'.h5'

with h5py.File(filename,'w') as f:
    f.create_dataset(data_key, data = data_full)
    f.create_dataset(scan_key, data = scan.positions)
    #Should add mask entry
    #Think about flux readout
    
print(filename)

/dls/e02/data/2019/cm22979-6/processing/Merlin/Merlin/20191001_15kV_ptycho/MoS2_700C/20191002 160536/pty_data_20191002_160536.h5
